In [1]:
"""
ChatPDF 简化版本
使用 LangChain 标准组件实现 PDF 文档问答系统
"""
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_qa_with_sources_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_community.llms import Tongyi
from typing import List, Dict, Tuple

# 配置参数
CHUNK_SIZE = 1000 
CHUNK_OVERLAP = 200 #重叠
EMBEDDING_MODEL = "text-embedding-v1"
LLM_MODEL = "qwen-turbo"
DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')

if not DASHSCOPE_API_KEY:
    raise ValueError("请设置 DASHSCOPE_API_KEY 环境变量")

def process_pdf(pdf_path: str) -> Tuple[List[str], List[Dict]]:
    """
    处理PDF文件，返回文本块和元数据
    """
    # 读取PDF
    reader = PdfReader(pdf_path)
    
    # 收集所有页面的文本和位置信息
    all_text = []
    page_positions = []  # 记录每个字符属于哪一页
    
    for page_num, page in enumerate(reader.pages, 1):
        page_text = page.extract_text()
        if not page_text:
            continue

        # 清理文本
        page_text = page_text.replace("百度文库", "").replace("好好学习，天天向上", "").strip()
        
        # 记录当前页面的所有字符位置
        for _ in range(len(page_text)):
            page_positions.append(page_num)
            
        all_text.append(page_text)
    
    # 合并所有文本
    full_text = "".join(all_text)
    
    # 使用LangChain的文本分割器
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", "。", "！", "？", ".", "!", "?", " ", ""],  # 添加中文标点
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        is_separator_regex=False
    )
    
    # 分割文本
    chunks = text_splitter.split_text(full_text)
    
    # 为每个文本块创建元数据
    chunk_metadata = []
    for chunk in chunks:
        # 找到这个chunk在原始文本中的位置
        start_pos = full_text.find(chunk)
        if start_pos == -1:
            # 如果找不到精确匹配，使用第一个字符的位置
            start_pos = 0
            
        # 获取这个chunk的起始页码
        start_page = page_positions[start_pos] if start_pos < len(page_positions) else 1
        
        # 获取这个chunk的结束页码
        end_pos = start_pos + len(chunk)
        end_page = page_positions[min(end_pos, len(page_positions)-1)] if end_pos < len(page_positions) else start_page
        
        # 如果chunk跨越了多页，使用起始页码
        chunk_metadata.append({
            "page": start_page,
            "source": pdf_path
        })
    
    return chunks, chunk_metadata

def create_vector_store(texts: List[str], metadata: List[Dict]):
    """
    创建向量存储
    """
    # 创建embeddings
    embeddings = DashScopeEmbeddings(
        model=EMBEDDING_MODEL,
        dashscope_api_key=DASHSCOPE_API_KEY
    )
    
    # 创建向量存储
    vector_store = FAISS.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadata
    )
    # print(vector_store)
    return vector_store



def create_qa_chain(vector_store):
    """
    创建问答链
    """
    # 创建LLM
    llm = Tongyi(
        model_name=LLM_MODEL,
        temperature=0.7,
        dashscope_api_key=DASHSCOPE_API_KEY
    )
    # 创建问答链
    qa_chain = load_qa_chain(
        llm=llm,
        chain_type="stuff",
    )
        
    
    return qa_chain, vector_store

In [2]:
# 处理PDF
# pdf_path = input("请输入PDF文件路径：")
pdf_path = './浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf'
texts, metadata = process_pdf(pdf_path)

# 创建向量存储
vector_store = create_vector_store(texts, metadata)

# 创建问答链
qa_chain, vector_store = create_qa_chain(vector_store)
vector_store

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20804\2280988212.py:124: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(


In [3]:


# 问答循环
# 问答循环
print("\n系统已准备就绪，可以开始提问了。输入'退出'结束对话。")
# while True:
#     question = input("\n请输入问题：")
question = "客户经理的职责是什么"
#     if question.lower() == '退出':
#         break
        
    # # 直接使用检索器获取相关文档，验证可以查出相关文档来
    # docs = vector_store.similarity_search(question, k=6)
    # print(f"\n检索到的文档数量：{len(docs)}")
    # if docs:
    #     print("检索到的文档内容：")
    #     for doc in docs:
    #         print(f"- {doc.page_content[:200]}...")
# 直接使用检索器获取相关文档，验证可以查出相关文档来
docs = vector_store.similarity_search(question, k=6)
print(f'docs={docs}')
#生成回答
result = qa_chain.run(
    input_documents=docs, 
    question=question,
)

print(f'result={result}')

# # 输出结果
# print("\n回答：")
# print(result)
# # 检查是否有相关文档且答案不是"不知道"
# if result and "不知道" not in result]:
#     print("\n来源：")
#     for doc in result:
#         print(f"- 页码：{doc.metadata.get('page', '未知')}, 来源：{doc.metadata.get('source', '未知')}")
# else:
#     print("\n未找到相关文档。")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20804\121758153.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(



系统已准备就绪，可以开始提问了。输入'退出'结束对话。
docs=[Document(id='0fd9398f-92d9-4ec8-9925-3040b1584243', metadata={'page': 1, 'source': './浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf'}, page_content='-   \n-1 上海浦东发展银行西安分行  \n个金客户经理管理考核暂行办法  \n \n \n第一章  总   则 \n第一条   为保证我分行个金客户经理制的顺利实施，有效调动个\n金客户经理的积极性，促进个金业务快速、稳定地发展，根据总行《上\n海浦东发展银行个人金融营销体系建设方案（试行）》要求，特制定\n《上海浦东发展银行西安分行个金客户经理管理考核暂行办法（试\n行）》（以下简称本办法）。  \n第二条   个金客户经理系指各支行（营业部）从事个人金融产品\n营销与市场开拓，为我行个人客户提供综合银行服务的我行市场人\n员。 \n第三条   考核内容分为二大类， 即个人业绩考核、 工作质量考核。\n个人业绩包括个人资产业务、负债业务、卡业务。工作质量指个人业\n务的资产质量。  \n第四条   为规范激励规则，客户经理的技术职务和薪资实行每年\n考核浮动。客户经理的奖金实行每季度考核浮动，即客户经理按其考\n核内容得分与行员等级结合，享受对应的行员等级待遇。-   \n-2 第二章  职位设置与职责  \n第五条   个金客户经理职位设置为：客户经理助理、客户经理、\n高级客户经理、资深客户经理。  \n第六条   个金客户经理的基本职责：  \n（一）   客户开发。研究客户信息、联系与选择客户、与客户建\n立相互依存、相互支持的业务往来关系，扩大业务资源，创造良好业\n绩； \n（二）业务创新与产品营销。把握市场竞争变化方向，开展市场\n与客户需 求的调研，对业务产品及服务进行创新；设计客户需求的产\n品组合、制订和实施市场营销方案；  \n（三）客户服务。负责我行各类表内外授信业务及中间业务的受\n理和运作，进行综合性、整体性的客户服务；  \n（四）防范风险，提高收益。提升风险防范意识及能力，提高经\n营产品质量；  \n（五）培养人材。在提高自身综合素质的同时，发扬团队精神，\n培养后备业务骨干。-   \n-3 